In [ ]:
using Pkg
Pkg.activate("..")

In [ ]:
using NCDatasets, Flux, CUDA, Plots, DelimitedFiles, Distributed, Dates, Measures
using Dates: format, now, value, Millisecond
import YAML
using BSON: @save, @load

gr(fmt =:png)

In [ ]:
# Set working directory to project base.
cd("..")
pwd()

In [ ]:
# Check cuda functionality
CUDA.functional()

This notebook is used as a first step towards the construction of a model. The use of a notebook makes it much easier to select data and construct a model. In this notebook we select the input and output, defines the model, create a config file. All of this is written to file in a new run-directory. 

To train the model we copy the script `train-model.jl` to the new run folder and runs the current version from that folder (loading config, model and data selection files).


# Selecting input and output.
The dataset is of relatively high resolution, hence it might be a good idea to reduce the dimensions.

In [ ]:
data_dir = "/data_large/stg/UMA_download/"; # Path to the folder with the scenario files.
train_data = "article_data/train_591/train.txt"; # Text file with a list of scenario files in the data_dir.
test_data = "article_data/bottom_UMAPS_shuf.txt";
grid_file = "/data_large/grids/Catania/C_CT.grd"; # Topography file at the inundation location.
runs_dir = "runs/"; # folder to store run within subfolder.
train_script ="src/train-model.jl";
ct_mask_file = "article_data/ct_mask.txt"; # Pixels target of predictions (true/false).
reg = 1e7/(591^3); # Parameter for l2 weight penalization in the loss function.
 
batch_size = 30;

## Selecting inundation area.

In [ ]:
ct_slice = (1:912,1:2224) # Subwindow of the inundation.
scale = (1,1) # Downsample dimensions - window of size scale is mapped to single pixel. Make sure you get integer dimension.
dims = (length(ct_slice[1]) ÷ scale[1],length(ct_slice[2]) ÷ scale[2]) # dimension of inundation map (matrix).
aspect_ratio = scale[1]/scale[2] # Pixel shape.

downsampler = MaxPool(scale)
upsampler = Upsample(scale)

topography = NCDataset(grid_file, "r")["z"][ct_slice...];

function downsample(x, scale)
    x = Array{Float32}(reshape(x, (size(topography)...,1,1)))
    return MaxPool(scale)(x)[:,:,1,1]
end

downsampled_topography = downsample(topography, scale);
ct_mask = downsample(BitArray(readdlm(ct_mask_file, '\t',Bool, '\n'))[ct_slice...], scale) |> BitArray;

In [ ]:
size(downsampled_topography) == dims

In [ ]:
# The maps may now be truncated according to the mask.
topo_truncated = zeros(Float32, dims);
topo_truncated[ct_mask] = downsampled_topography[ct_mask];

p1 = heatmap(
    downsampled_topography'; 
    aspect_ratio=1/aspect_ratio, 
    xlim=(1,dims[1]), 
    ylim=(1,dims[2]),
    c=:oleron, 
    clims = (-30,30),
    margins= 3mm
)
p2 = heatmap(
    topo_truncated'; 
    aspect_ratio=1/aspect_ratio, 
    xlim=(1,dims[1]), 
    ylim=(1,dims[2]),
    c=:oleron, 
    clims = (-30,30),
    margins= 3mm
)

p3 = heatmap(
    ct_mask'; 
    aspect_ratio=1/aspect_ratio, 
    xlim=(1,dims[1]), 
    ylim=(1,dims[2]),
    margins = 3mm
)

plot(
    p1, p2, p3,
    layout = (1,3), 
    size=(1000,600), 
    title=["Topography" "Truncated Topography" "Mask"], 
    titleloc= :center
)

In [ ]:
sum(ct_mask) # Number of pixels for prediction.

# Timeseries data

Inspection of timeseries data.

In [ ]:
# Selecting arbitrary scenario file for inspection.
scenario_ts = "10_PS_2803/2054_E02020N3739E02658N3366-PS-Str_PYes_Hom-M888_E02122N3652_S004_ts.nc"
isfile(joinpath(data_dir, scenario_ts))

In [ ]:
joinpath(data_dir, scenario_ts) # Complete path to the scenario

In [ ]:
# Load the dataset
data_ts = NCDataset(joinpath(data_dir, scenario_ts), "r")

In [ ]:
ts_slice = 30:45,1:480 # gauge number, time
data_ts["eta"][ts_slice...]

In [ ]:
# plot the 
t_0 = data_ts["time"][ts_slice[2][1]]
time_scale = [round((t-t_0)/Millisecond(1000*60)) for t in data_ts["time"][ts_slice[2]]]
p = heatmap(data_ts["eta"][ts_slice...], 
    c=:deepsea, 
    legend=:none, 
    xlabel="Time [Minutes]", 
    xticks=(ts_slice[2][1:40:480], time_scale[1:40:480]), 
    xrotation = 45,
    ylabel="POI's", 
    yticks=(Array(1:2:16), Array(30:2:46))
)
display(p)
#savefig("timeseries_heatmap.png")

# Reading data

Create datareader first, in order to test the model.

In [ ]:
# Add process for dataloading.
addprocs(1; exeflags="--project")

In [ ]:
@everywhere include("scripts/datareader.jl")

In [ ]:
config = Dict(
    "data_dir" => data_dir,
    "train_data" => train_data,
    "test_data" => test_data,
    "grid_file" => grid_file,
    "batch_size" => batch_size,
    "ct_slice" => ct_slice,
    "ts_slice" => ts_slice,
    "scale" => scale, 
    "reg" => reg,
)

In [ ]:
@everywhere begin
    reader = DataReader.Reader($config)
end

In [ ]:
scenarios = DataReader.scenarios(train_data)

In [ ]:
for i in 1:100
    scenario = take!(scenarios)
    @info "Scenario $i is $scenario"
end

@info "Load batches with scenarios."
batches = RemoteChannel(()->Channel(4))

for worker in workers()
    remote_do(reader, worker, scenarios, batches)
end

batch = take!(batches);

In [ ]:
batch.scenario_names

In [ ]:
batch.flow_depths |> size

In [ ]:
batch.deformed_topographies |> size

In [ ]:
batch.etas |> size

In [ ]:
p1 = heatmap(
    batch.flow_depths[:,:,1,1]', 
    aspect_ratio=1/aspect_ratio,
    xlim=(1,dims[1]), 
    ylim=(1,dims[2])
)
p2 = heatmap(
    batch.deformed_topographies[:,:,1,1]', 
    aspect_ratio=1/aspect_ratio,
    c=:oleron, 
    clims = (-30,30),
    xlim=(1,dims[1]), 
    ylim=(1,dims[2])
)

plot(
    p1, p2, 
    layout = (1,2), 
    size=(1200,700), 
    title=["Flow Depth" "Deformed Topography"], 
    titleloc= :center
)

## Defining model in a separate file.

There are issues with storing more complex models using BSON.jl. Teherfore we use a library to specifically write weights to file. This means we have to define model structure in a separate file.

In [ ]:
include("scripts/model_config.jl")

model = ModelConfig.get_model()

In [ ]:
model_name = "mc32_l16_rel"

## Optimizer

In [ ]:
opt = ADAM()

# Create rundir with appropriate config.

In [ ]:
function make_run_dir()
    timestamp = format(now(), "YYYYmmdd-HHMMSS")
    #dir_name = joinpath(runs_dir, model_name*"_$timestamp")
    dir_name = joinpath(runs_dir, model_name)
    @assert !ispath(dir_name) "Output directory already exists"
    mkpath(dir_name)
    return dir_name
end

In [ ]:
rundir = make_run_dir()
#rundir = "article_runs/stridedConv_20230308-174706/"

In [ ]:
# write mask to file.
open(joinpath(rundir, "ct_mask.txt"), "w") do io
           writedlm(io, ct_mask)
end

In [ ]:
config = Dict(
    "data_dir" => data_dir,
    "train_data" => joinpath(pwd(), train_data),
    "test_data" => joinpath(pwd(), test_data),
    "grid_file" => grid_file,
    "model_name" => model_name,
    "rundir" => joinpath(pwd(), rundir),
    "batch_size" => batch_size,
    "ct_slice" => ct_slice,
    "scale" => scale,
    "dims" => dims,
    "ts_slice" => ts_slice,
    "max_batch_nr" => 20000,
    "gpu" => true,
    "reg" => reg,
)

In [ ]:
YAML.write_file(joinpath(rundir, "config.yml"), config)

In [ ]:
@save joinpath(rundir, "optimizer.bson") opt

In [ ]:
ModelConfig.save(model, joinpath(rundir,"$(model_name).jls"))

In [ ]:
# Copy scripts for training to rundir. Next run the script 
cp("scripts/train-model.jl", joinpath(rundir,"train-model.jl"), force=true)
cp("scripts/datareader.jl", joinpath(rundir, "datareader.jl"), force=true)
cp("scripts/model_config.jl", joinpath(rundir, "model_config.jl"), force=true)

In [ ]:
@info config

Next, to run the training, run the script from the new run directory
```terminal
[rundir]$ julia --project train-model.jl
```
